# EAS 587 Quiz 1 Practice - Building Permits Analysis

This notebook provides practice for Quiz 1 focusing on data analysis and K-means clustering techniques.

**Time Target: 20 minutes**

## Dataset
Buffalo building permits data from 2015 to present, containing information about:
- Permit types (electrical, plumbing, heating, construction, etc.)
- Geographic locations
- Fees and work values
- Contractor information
- Neighborhoods and districts

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('default')

In [ ]:
# Load the dataset
url = 'https://raw.githubusercontent.com/delveccj/EAS_587_Quiz1_Practice/main/data/permits_2015_present.csv'
df = pd.read_csv(url)

print(f"Dataset shape: {df.shape}")
df.head()

## Quiz Questions

Complete the following questions by writing code in the cells below each question.

🔍 **Question 1:** Show the shape of this dataframe

In [ ]:
# Answer:
df.shape

🔍 **Question 2:** Display the first 10 rows

In [ ]:
# Answer
df.head(10)

🔍 **Question 3:** Check for missing values in the dataset. Display the count of missing values and percentage missing for each column that has missing data. Format the output as a DataFrame with columns 'Missing Count' and 'Missing Percentage'.

In [ ]:
# Answer
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_summary = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percentage
})

# Show only columns with missing values
missing_summary[missing_summary['Missing Count'] > 0]

🔍 **Question 4:** Drop columns that are missing more than 90% of their data

In [ ]:
# Answer
# Calculate percentage missing for each column
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Find columns with more than 90% missing data
columns_to_drop = missing_percentage[missing_percentage > 90].index.tolist()
print("Columns to drop (>90% missing):", columns_to_drop)

# Drop the columns
df_cleaned = df.drop(columns=columns_to_drop)
print(f"Original shape: {df.shape}")
print(f"After dropping columns: {df_cleaned.shape}")

🔍 **Question 5:** Remove rows where both Latitude and Longitude are missing (since we'll need coordinates for clustering)

In [ ]:
# Answer
# Remove rows where both Latitude and Longitude are missing
df_cleaned = df_cleaned.dropna(subset=['Latitude', 'Longitude'])

print(f"After removing rows with missing coordinates: {df_cleaned.shape}")
print(f"Rows removed: {df.shape[0] - df_cleaned.shape[0]}")

🔍 **Question 6:** Look at the Fees column. What data cleaning operation do you think needs to be performed? Apply that operation.

In [ ]:
# Answer
# Clean Fees column - remove $ and convert to numeric
df_cleaned['Fees'] = df_cleaned['Fees'].str.replace('$', '').str.replace(',', '')
df_cleaned['Fees'] = pd.to_numeric(df_cleaned['Fees'], errors='coerce')

print("Fees column cleaned:")
print(df_cleaned['Fees'].describe())

🔍 **Question 7:** Based on Question 6 - what other column would you need to clean as well? Go ahead and clean it now.

In [ ]:
# Answer
# Clean Value of Work column - same issue as Fees (has $ and commas)
df_cleaned['Value of Work'] = df_cleaned['Value of Work'].str.replace('$', '').str.replace(',', '')
df_cleaned['Value of Work'] = pd.to_numeric(df_cleaned['Value of Work'], errors='coerce')

print("Value of Work column cleaned:")
print(df_cleaned['Value of Work'].describe())

🔍 **Question 8:** Create tabular histogram counts that show the number of permits in ranges: '$0-1K', '$1K-5K', '$5K-25K', '$25K-100K', '$100K-500K', '$500K+'

In [ ]:
# Answer
# Show distribution in value ranges (more informative than histogram)
vow = df_cleaned['Value of Work'].dropna()

# Create value range bins
bins = [0, 1000, 5000, 25000, 100000, 500000, np.inf]
labels = ['$0-1K', '$1K-5K', '$5K-25K', '$25K-100K', '$100K-500K', '$500K+']

# Count permits in each range
value_ranges = pd.cut(vow, bins=bins, labels=labels, include_lowest=True)
range_counts = value_ranges.value_counts().sort_index()

print("Value of Work Distribution:")
print(range_counts)
print(f"\nTop 10 highest value projects:")
print(df_cleaned.nlargest(10, 'Value of Work')[['Address', 'Permit Type', 'Value of Work', 'Description of Work']])

🔍 **Question 9:** Create a new dataframe called `df_kmeans` for clustering analysis. Remove permits with Value of Work over $500,000 and keep only rows where Value of Work and Fees are not null.

In [ ]:
# Answer
# Create dataframe for K-means clustering
df_kmeans = df_cleaned.copy()

# Remove high-value outliers (>$500K)
df_kmeans = df_kmeans[df_kmeans['Value of Work'] <= 500000]

# Keep only rows with complete data for clustering features
df_kmeans = df_kmeans.dropna(subset=['Value of Work', 'Fees'])

print(f"Original cleaned data: {df_cleaned.shape}")
print(f"K-means ready data: {df_kmeans.shape}")
print(f"Removed {df_cleaned.shape[0] - df_kmeans.shape[0]} rows")

🔍 **Question 10:** Perform an elbow analysis to determine the optimal number of clusters for contractor and Value of Work clustering. Plot the Within-Cluster Sum of Squares (WCSS) for k values from 1 to 10 using Value of Work and Contractor License Number as features.

In [ ]:
# Answer
# Prepare data for elbow analysis
df_contractor = df_kmeans.dropna(subset=['Contractor License Number']).copy()

# Encode contractor license numbers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_contractor['Contractor_Encoded'] = le.fit_transform(df_contractor['Contractor License Number'])

# Prepare features for clustering
features = ['Value of Work', 'Contractor_Encoded']
X = df_contractor[features].copy()

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform elbow analysis
wcss = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(k_range, wcss, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.grid(True)
plt.show()

print("WCSS values for each k:")
for k, w in zip(k_range, wcss):
    print(f"k={k}: WCSS={w:,.0f}")

🔍 **Question 11:** Based on your elbow analysis, perform K-means clustering using Value of Work and Contractor License Number. Use the optimal number of clusters from Question 10 and provide a detailed analysis showing the characteristics of each cluster (average values, top contractors, license types).

In [ ]:
# Answer
# Prepare data for clustering with contractors
df_contractor = df_kmeans.dropna(subset=['Contractor License Number']).copy()

# Encode contractor license numbers (categorical to numeric)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_contractor['Contractor_Encoded'] = le.fit_transform(df_contractor['Contractor License Number'])

# Prepare features for clustering
features = ['Value of Work', 'Contractor_Encoded']
X = df_contractor[features].copy()

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply K-means clustering
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# Add cluster labels to dataframe
df_contractor['Cluster'] = clusters

print(f"Clustering complete with {len(df_contractor)} permits")
print(f"Number of unique contractors: {df_contractor['Contractor License Number'].nunique()}")
print(f"Cluster distribution:\n{pd.Series(clusters).value_counts().sort_index()}")

print("CLUSTER ANALYSIS BY VALUE OF WORK AND CONTRACTOR PATTERNS:\n")

for cluster_id in range(5):
    cluster_data = df_contractor[df_contractor['Cluster'] == cluster_id]
    
    print(f"CLUSTER {cluster_id} ({len(cluster_data)} permits):")
    print(f"  Average Value of Work: ${cluster_data['Value of Work'].mean():,.2f}")
    print(f"  Median Value of Work: ${cluster_data['Value of Work'].median():,.2f}")
    print(f"  Most common License Type: {cluster_data['License Type'].mode().iloc[0] if not cluster_data['License Type'].mode().empty else 'N/A'}")
    
    # Top contractors in this cluster by permit count
    top_contractors = cluster_data.groupby(['Contractor License Number', 'Applicant']).size().reset_index(name='permit_count')
    top_contractors = top_contractors.sort_values('permit_count', ascending=False).head(3)
    
    print(f"  Top 3 contractors by permit count:")
    for i, (idx, row) in enumerate(top_contractors.iterrows(), 1):
        print(f"    {i}. {row['Applicant']} (License: {row['Contractor License Number']}) - {row['permit_count']} permits")
    
    # Value of Work distribution
    print(f"  Value range: ${cluster_data['Value of Work'].min():,.2f} - ${cluster_data['Value of Work'].max():,.2f}")
    print("=" * 60)